In [ ]:
# Импортируем библиотеки
import os
import pandas as pd
import numpy as np
import re

import time
# selenium 4
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from geopy.geocoders import Nominatim
import plotly.express as px
import plotly.graph_objects as go

from func.myfunc import *


In [ ]:
# Задаем путь к дериктории с данными
folder_path = './data/fin'

data = pd.DataFrame()
# В цикле перебираем файлы в папке и добавляем их в датафрейм
# с присвоением новых индексов и добавляем имя файла как отдельный
# признак
for file in os.scandir(folder_path):
    if file.is_file():
        df = pd.read_csv(file)
        df['file_name'] = file.name
        data = pd.concat([data, df], ignore_index=True)

# Проверяем результат
display(data.head())


,info,link,url,head,owner,characters,comment,options,parse_date,file_name
0,Hyundai Custin\n1.5 л / 170 л.с. / Бензин\nавт...,"<div class=""ListingItem__description""><div cla...",https://auto.ru/cars/used/sale/hyundai/custin/...,"Hyundai Custin, 2024\n15 сентября\n137 (5 сего...",Алексей АВТОНАПРАВЛЕНИЕ\nНет отзывов\nЕкатерин...,Наличие\nВ наличии\nГод выпуска\n2024\nПробег\...,Регистрация в ГИБДД без очереди и получение Го...,Безопасность\n•\nПодушка безопасности водителя...,20-09-2024,chel_data.csv
1,Hyundai Solaris II\n1.6 л / 123 л.с. / Бензин\...,"<div class=""ListingItem__description""><div cla...",https://auto.ru/cars/used/sale/hyundai/solaris...,"Hyundai Solaris, 2019\n19 сентября\n9 (4 сегод...",Автомобили с пробегом АГАТ на Шефской\n Проспе...,Наличие\nВ наличии\nПоколение\nII\nГод выпуска...,Hyundai Solaris 2019 c пробегом прошел комплек...,Салон\n•\nКоличество мест: 5,20-09-2024,chel_data.csv
2,Dongfeng DF6\n2.3 л / 163 л.с. / Дизель\nавтом...,"<div class=""ListingItem__description""><div cla...",https://auto.ru/cars/used/sale/dongfeng/df6/11...,"Dongfeng DF6, 2022\nДунфэн Ди-Эф шесть\n2 сент...",Автомобили с пробегом АГАТ на Шефской\n Проспе...,Наличие\nВ наличии\nГод выпуска\n2022\nПробег\...,DongFeng DF6 2022 c пробегом прошел комплексну...,Салон\n•\nКоличество мест: 4,20-09-2024,chel_data.csv
3,Porsche Cayman S II (981)\n3.4 л / 325 л.с. / ...,"<div class=""ListingItem__description""><div cla...",https://auto.ru/cars/used/sale/porsche/cayman/...,"Porsche Cayman S, 2013\n16 сентября\n583 (18 с...","Частное лицо\nЕкатеринбургМосковская улица, 192",Наличие\nВ наличии\nПоколение\nII (981)\nГод в...,Автомобиль в идеальном состоянии\nСделано то\n...,Безопасность\n•\nПодушка безопасности водителя...,20-09-2024,chel_data.csv
4,Volkswagen Passat CC I Рестайлинг\n1.8 л / 152...,"<div class=""ListingItem__description""><div cla...",https://auto.ru/cars/used/sale/volkswagen/pass...,"Volkswagen Passat CC, 2012\n19 сентября\n64 (1...",Владимир\nЧелябинскплощадь Ленина,Наличие\nВ наличии\nПоколение\nI Рестайлинг\nГ...,"По кузову живая, сцепление новое, моховик новы...",Безопасность\n•\nПодушка безопасности водителя...,20-09-2024,chel_data.csv


In [ ]:
# Посмотрим на наличие пропущенных значений
print(data.info())

# Видно, что среди обязательных полей (поля comment и options могут быть
# не заполнены продавцом при создании объявления) максимальное число пропусков
# в поле head. Скорее всего такие карточки на момент сбора информации были уже
# удалены.
mask = data['head'].isna()
display(data[mask])

# Избавимся от таких строк, а также переназначим индексы.
data = data.dropna(subset=['head']).reset_index(drop='index')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40843 entries, 0 to 40842
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   info        39911 non-null  object
 1   link        40843 non-null  object
 2   url         40843 non-null  object
 3   head        39885 non-null  object
 4   owner       39885 non-null  object
 5   characters  39885 non-null  object
 6   comment     38730 non-null  object
 7   options     38428 non-null  object
 8   parse_date  40843 non-null  object
 9   file_name   40843 non-null  object
dtypes: object(10)
memory usage: 3.1+ MB
None


,info,link,url,head,owner,characters,comment,options,parse_date,file_name
18,NaN,"<div class=""ListingItem__description""><div cla...",https://auto.ru/cars/used/sale/nissan/qashqai/...,NaN,NaN,NaN,NaN,NaN,20-09-2024,chel_data.csv
61,NaN,"<div class=""ListingItem__description""><div cla...",https://auto.ru/cars/used/sale/nissan/qashqai/...,NaN,NaN,NaN,NaN,NaN,20-09-2024,chel_data.csv
119,NaN,"<div class=""ListingItem__description""><div cla...",https://auto.ru/cars/used/sale/vaz/vesta/11247...,NaN,NaN,NaN,NaN,NaN,20-09-2024,chel_data.csv
154,NaN,"<div class=""ListingItem__description""><div cla...",https://auto.ru/cars/used/sale/renault/duster/...,NaN,NaN,NaN,NaN,NaN,20-09-2024,chel_data.csv
283,NaN,"<div class=""ListingItem__description""><div cla...",https://auto.ru/cars/used/sale/hyundai/elantra...,NaN,NaN,NaN,NaN,NaN,20-09-2024,chel_data.csv
...,...,...,...,...,...,...,...,...,...,...
40562,NaN,"<div class=""ListingItem__description""><div cla...",https://auto.ru/cars/used/sale/ford/focus/1124...,NaN,NaN,NaN,NaN,NaN,21-09-2024,vol_data.csv
40590,NaN,"<div class=""ListingItem__description""><div cla...",https://auto.ru/cars/used/sale/vaz/2170/112422...,NaN,NaN,NaN,NaN,NaN,21-09-2024,vol_data.csv
40665,NaN,"<div class=""ListingItem__description""><div cla...",https://auto.ru/cars/used/sale/volkswagen/toua...,NaN,NaN,NaN,NaN,NaN,21-09-2024,vol_data.csv
40740,Chevrolet Lacetti I\n1.6 л / 109 л.с. / Бензин...,"<div class=""ListingItem__description""><div cla...",https://auto.ru/cars/used/sale/chevrolet/lacet...,NaN,NaN,NaN,NaN,NaN,21-09-2024,vol_data.csv


In [ ]:
# Достанем полезную информацию из поля характеристик. Найдем строку с
# максимальным количеством символов в поле характеристик и выведем её
# значение
mask = data['characters'].str.len() == data['characters'].str.len().max()
display(data[mask]['characters'])

# Видно, что поле содержит название характеристики и её значение разделенные
# знаком табуляции. Достанем все названия характеристик. Последнее значение
# "показать в отчете" нам не требуется - это кнопка на платную услугу и при
# сборе информации мы ей не пользовались.
characters_list = data[mask]['characters'].str.split('\n').to_list()[0][::2]
display(characters_list)


19107    Наличие\nВ наличии\nПоколение\n150 Series Рест...
Name: characters, dtype: object

['Наличие',
 'Поколение',
 'Год выпуска',
 'Пробег',
 'Кузов',
 'Цвет',
 'Двигатель',
 'Комплектация',
 'Налог',
 'Коробка',
 'Привод',
 'Руль',
 'Состояние',
 'Владельцы',
 'ПТС',
 'Владение',
 'Таможня',
 'Обмен',
 'Гарантия',
 'VIN',
 'Госномер',
 'Показать в отчёте']

In [ ]:
# Сформируем датасет с характеристиками автомобилей, применив нашу функцию
# # из модуля myfunc к исходным данным
df = pd.DataFrame(data['characters'].apply(get_characters).to_list(),
                  columns=['availability',
                           'generation',
                           'year',
                           'mileage',
                           'body',
                           'color',
                           'engine',
                           'equipment',
                           'tax',
                           'transmission',
                           'drive',
                           'steering_wheel',
                           'condition',
                           'owners',
                           'pts',
                           'possession',
                           'customs',
                           'exchange',
                           'vin',
                           'state_number'
                           ]
                  )

# Добавим эти данные к исходному датасету и проверим результат
data = data.join(df)
display(data.head())


,info,link,url,head,owner,characters,comment,options,parse_date,file_name,...,drive,steering_wheel,condition,owners,pts,possession,customs,exchange,vin,state_number
0,Hyundai Custin\n1.5 л / 170 л.с. / Бензин\nавт...,"<div class=""ListingItem__description""><div cla...",https://auto.ru/cars/used/sale/hyundai/custin/...,"Hyundai Custin, 2024\n15 сентября\n137 (5 сего...",Алексей АВТОНАПРАВЛЕНИЕ\nНет отзывов\nЕкатерин...,Наличие\nВ наличии\nГод выпуска\n2024\nПробег\...,Регистрация в ГИБДД без очереди и получение Го...,Безопасность\n•\nПодушка безопасности водителя...,20-09-2024,chel_data.csv,...,передний,Левый,Не требует ремонта,1 владелец,Оригинал,None,Растаможен,Рассмотрю варианты,MXH**************,None
1,Hyundai Solaris II\n1.6 л / 123 л.с. / Бензин\...,"<div class=""ListingItem__description""><div cla...",https://auto.ru/cars/used/sale/hyundai/solaris...,"Hyundai Solaris, 2019\n19 сентября\n9 (4 сегод...",Автомобили с пробегом АГАТ на Шефской\n Проспе...,Наличие\nВ наличии\nПоколение\nII\nГод выпуска...,Hyundai Solaris 2019 c пробегом прошел комплек...,Салон\n•\nКоличество мест: 5,20-09-2024,chel_data.csv,...,передний,Левый,Не требует ремонта,1 владелец,Оригинал,None,Растаможен,Рассмотрю варианты,Z94**************,None
2,Dongfeng DF6\n2.3 л / 163 л.с. / Дизель\nавтом...,"<div class=""ListingItem__description""><div cla...",https://auto.ru/cars/used/sale/dongfeng/df6/11...,"Dongfeng DF6, 2022\nДунфэн Ди-Эф шесть\n2 сент...",Автомобили с пробегом АГАТ на Шефской\n Проспе...,Наличие\nВ наличии\nГод выпуска\n2022\nПробег\...,DongFeng DF6 2022 c пробегом прошел комплексну...,Салон\n•\nКоличество мест: 4,20-09-2024,chel_data.csv,...,полный,Левый,Не требует ремонта,2 владельца,Оригинал,None,Растаможен,Рассмотрю варианты,LJN**************,None
3,Porsche Cayman S II (981)\n3.4 л / 325 л.с. / ...,"<div class=""ListingItem__description""><div cla...",https://auto.ru/cars/used/sale/porsche/cayman/...,"Porsche Cayman S, 2013\n16 сентября\n583 (18 с...","Частное лицо\nЕкатеринбургМосковская улица, 192",Наличие\nВ наличии\nПоколение\nII (981)\nГод в...,Автомобиль в идеальном состоянии\nСделано то\n...,Безопасность\n•\nПодушка безопасности водителя...,20-09-2024,chel_data.csv,...,задний,Левый,Не требует ремонта,3 или более,Оригинал,None,Растаможен,None,WP0**************,******|196
4,Volkswagen Passat CC I Рестайлинг\n1.8 л / 152...,"<div class=""ListingItem__description""><div cla...",https://auto.ru/cars/used/sale/volkswagen/pass...,"Volkswagen Passat CC, 2012\n19 сентября\n64 (1...",Владимир\nЧелябинскплощадь Ленина,Наличие\nВ наличии\nПоколение\nI Рестайлинг\nГ...,"По кузову живая, сцепление новое, моховик новы...",Безопасность\n•\nПодушка безопасности водителя...,20-09-2024,chel_data.csv,...,передний,Левый,Не требует ремонта,3 или более,Оригинал,None,Растаможен,Рассмотрю варианты,WVW**************,******|174


In [ ]:
# Признак 'engine' содержит информацию о мощности, объеме и типе двигателя.
# Создадим вспомогательный датасет, разделив этот признак на составляющие с
# помощьюю функции определенной в модуле myfunc
df = pd.DataFrame(data['engine'].apply(get_engine).to_list(),
                  columns=['eng_hpower', 'eng_vol', 'eng_type'])

# Добавим эти данные к исходному датасету
data = data.join(df)


In [ ]:
# В поле info нас интересует цена продажи, марка и модель машины а также
# город, в котором размещалось объявление. Цена всегда располагается на 7
# позиции в поле info, для ее извлечения воспользуемся лямбда функцией.
# Получив строку на 7 позиции разделим её по пробелу в список, соединим его
# элементы обратно в строку, чтобы избавится от пробелов и преобразуем в число
data['price'] = data['info'].map(lambda x: int(''.join(x.split('\n')[6][:-1]
                                                       .split(' ')
                                                       )
                                               )
                                 )

# Марка машины, её модель и поколение находятся на 0 позиции в поле info.
# Для извлечения марки автомобиля применим нашу функцию get_brand из модуля
# myfunc
data['brand'] = data['info'].map(get_brand)

# Чтобы получить наименование модели проще всего будет убрать из нулевого
# элемента поля info информацию о марке и поколении автомобиля. Воспользуемся
# для этого нашей функцией из модуля myfunc
data['model'] = data.apply(get_model, axis=1)


In [ ]:
# Наименование города будем получать из наименования файла. Создадим словарь с
# ключами из наименований файлов и значениями из наименований городов
sity_dict = {'chel_data.csv': 'Челябинск',
             'ekat_data.csv': 'Екатеринбург',
             'hab_data.csv': 'Хабаровск',
             'krasnk_data.csv': 'Красноярск',
             'krasnodar_data.csv': 'Краснодар',
             'mos_data.csv': 'Москва',
             'nal_data.csv': 'Нальчик',
             'nnov_data.csv': 'Нижний Новгород',
             'novsib.csv': 'Новосибирск',
             'pyat_data.csv': 'Пятигорск',
             'rost_data.csv': 'Ростов-на-Дону',
             'sam_data.csv': 'Самара',
             'spb_data.csv': 'Санкт-Петербург',
             'tula_data.csv': 'Тула',
             'vlad_data.csv': 'Владивосток',
             'vol_data.csv': 'Вологда'
             }

# Применим лямбда-функцию для создания признака
data['sity'] = data['file_name'].map(lambda x: sity_dict[x])

# В некоторых объявлениях в поле "characters" отсутствуют сведенья о пробеге.
# Попытаемся достать эти сведенья из поля "info". Для это определим функцию.
# Применим функцию restore_mileag из модуля myfunc для восстановления
# пропущенных значений пробега автомобиля
data['mileage'] = data.apply(restore_mileage, axis=1)

# Преобразуем пробег в числа убрав лишние пробелы между разрядами и
# размерность "км"
data['mileage'] = data['mileage'].map(lambda x: int(''.join(x
                                                            .split(' ')[:-1]
                                                            )
                                                    )
                                      )

# Проверим результат
display(data.head())


,info,link,url,head,owner,characters,comment,options,parse_date,file_name,...,exchange,vin,state_number,eng_hpower,eng_vol,eng_type,price,brand,model,sity
0,Hyundai Custin\n1.5 л / 170 л.с. / Бензин\nавт...,"<div class=""ListingItem__description""><div cla...",https://auto.ru/cars/used/sale/hyundai/custin/...,"Hyundai Custin, 2024\n15 сентября\n137 (5 сего...",Алексей АВТОНАПРАВЛЕНИЕ\nНет отзывов\nЕкатерин...,Наличие\nВ наличии\nГод выпуска\n2024\nПробег\...,Регистрация в ГИБДД без очереди и получение Го...,Безопасность\n•\nПодушка безопасности водителя...,20-09-2024,chel_data.csv,...,Рассмотрю варианты,MXH**************,None,170,1.5 л,Бензин,4300000,Hyundai,Custin,Челябинск
1,Hyundai Solaris II\n1.6 л / 123 л.с. / Бензин\...,"<div class=""ListingItem__description""><div cla...",https://auto.ru/cars/used/sale/hyundai/solaris...,"Hyundai Solaris, 2019\n19 сентября\n9 (4 сегод...",Автомобили с пробегом АГАТ на Шефской\n Проспе...,Наличие\nВ наличии\nПоколение\nII\nГод выпуска...,Hyundai Solaris 2019 c пробегом прошел комплек...,Салон\n•\nКоличество мест: 5,20-09-2024,chel_data.csv,...,Рассмотрю варианты,Z94**************,None,123,1.6 л,Бензин,1365000,Hyundai,Solaris,Челябинск
2,Dongfeng DF6\n2.3 л / 163 л.с. / Дизель\nавтом...,"<div class=""ListingItem__description""><div cla...",https://auto.ru/cars/used/sale/dongfeng/df6/11...,"Dongfeng DF6, 2022\nДунфэн Ди-Эф шесть\n2 сент...",Автомобили с пробегом АГАТ на Шефской\n Проспе...,Наличие\nВ наличии\nГод выпуска\n2022\nПробег\...,DongFeng DF6 2022 c пробегом прошел комплексну...,Салон\n•\nКоличество мест: 4,20-09-2024,chel_data.csv,...,Рассмотрю варианты,LJN**************,None,163,2.3 л,Дизель,2899000,Dongfeng,DF6,Челябинск
3,Porsche Cayman S II (981)\n3.4 л / 325 л.с. / ...,"<div class=""ListingItem__description""><div cla...",https://auto.ru/cars/used/sale/porsche/cayman/...,"Porsche Cayman S, 2013\n16 сентября\n583 (18 с...","Частное лицо\nЕкатеринбургМосковская улица, 192",Наличие\nВ наличии\nПоколение\nII (981)\nГод в...,Автомобиль в идеальном состоянии\nСделано то\n...,Безопасность\n•\nПодушка безопасности водителя...,20-09-2024,chel_data.csv,...,None,WP0**************,******|196,325,3.4 л,Бензин,5400000,Porsche,Cayman S,Челябинск
4,Volkswagen Passat CC I Рестайлинг\n1.8 л / 152...,"<div class=""ListingItem__description""><div cla...",https://auto.ru/cars/used/sale/volkswagen/pass...,"Volkswagen Passat CC, 2012\n19 сентября\n64 (1...",Владимир\nЧелябинскплощадь Ленина,Наличие\nВ наличии\nПоколение\nI Рестайлинг\nГ...,"По кузову живая, сцепление новое, моховик новы...",Безопасность\n•\nПодушка безопасности водителя...,20-09-2024,chel_data.csv,...,Рассмотрю варианты,WVW**************,******|174,152,1.8 л,Бензин,1050000,Volkswagen,Passat CC,Челябинск


In [ ]:
# Не для всех автомобилей посчитан транспортный налог.
print(f"Количество пропусков в призенаке tax: "
      f"{data[data['tax'] == 'None'].shape[0]}")

# Код парсера, приведенный ниже закоментирован во избежании случайных запусков
# # Попробуем это исправить с помощью онлайн сервиса по расчету налога.
# # Создаем список индексов строк в которых отсутствует налог.
# ind_ls = list(data[data['tax'] == 'None'].index)

# # Кроме того нам потребуется наименование региона и текущий год
# year = '2024'
# loc_dict = {'ekat_data.csv': 'сверд',
#             'hab_data.csv': 'хаб',
#             'krasnk_data.csv': 'краснояр',
#             'mos_data.csv': 'москва',
#             'pyat_data.csv': 'ставроп',
#             'rost_data.csv': 'ростов',
#             'sam_data.csv': 'самар',
#             'spb_data.csv': 'санкт',
#             'vlad_data.csv': 'приморск',
#             'chel_data.csv': 'челяб',
#             'krasnodar_data.csv': 'краснод',
#             'nal_data.csv': 'кабар',
#             'nnov_data.csv': 'ниж',
#             'novsib.csv': 'новосиб',
#             'tula_data.csv': 'тул',
#             'vol_data.csv': 'волог'
#             }

# # запускаем вебдрайвер и передаем в него ссылку на страницу с калькулятором
# driver = webdriver.Chrome()
# url = 'https://www.nalog.gov.ru/rn77/service/calc_transport/'
# driver.get(url=url)

# #  В цикле перебираем индексы по текущему срезу списка
# for ind in ind_ls:
#     try:
#         # Определяем регион для текущего индекса
#         region = loc_dict[data.loc[ind, 'file_name']]

#         # Задаем ожидание загрузки поля с выбором региона
#         element = WebDriverWait(
#             driver,
#             10
#             ).until(
#                 EC.presence_of_element_located(
#                     (By.ID, "select2-ctl00_ctl00_ctl00_ddlRegion-container")
#                     )
#                 )

#         # Для Владивостока отсутствует информация для автомобилей с мощностью
#         # свыше 500 л.с., примем для таких автомобилей налог для Москвы
#         if (data.loc[ind, 'file_name'] == 'vlad_data.csv') and \
#            (data.loc[ind, 'eng_hpower'] > 500):
#             region = 'москва'

#         # Находим это поле и как только оно становиться кликабельным кликаем
#         # по нему
#         reg_select_field = driver.find_element(
#             By.ID,
#             "select2-ctl00_ctl00_ctl00_ddlRegion-container"
#             )
#         reg_select = WebDriverWait(driver, 10).until(
#             EC.element_to_be_clickable(reg_select_field)
#             )
#         reg_select.click()

#         # Ждем появления строки поиска (строка поиска и выпадающий список с
#         # вариантами)
#         element = WebDriverWait(driver, 10).until(
#             EC.presence_of_element_located(
#                 (By.CLASS_NAME, 'select2-search__field')
#                 )
#             )

#         # Находим строку поиска, вписываем наименование/часть наименования
#         # региона и нажимаем ввод
#         reg_select = driver.find_element(By.CLASS_NAME,
#                                          'select2-search__field'
#                                          )
#         reg_select.send_keys(region)
#         reg_select.send_keys(Keys.RETURN)

#         # Повторяем эту процедуру для поля с выбором текущего года
#         element = WebDriverWait(
#             driver,
#             10
#             ).until(
#                 EC.presence_of_element_located(
#                     (By.ID, "select2-ctl00_ctl00_ctl00_ddlYear-container"
#                      )
#                     )
#                 )
#         year_select_field = driver.find_element(
#             By.ID,
#             "select2-ctl00_ctl00_ctl00_ddlYear-container"
#             )
#         year_select = WebDriverWait(driver, 10).until(
#             EC.element_to_be_clickable(year_select_field)
#             )
#         year_select.click()
#         element = WebDriverWait(driver, 10).until(
#             EC.presence_of_element_located((By.CLASS_NAME,
#                                             'select2-search__field'
#                                             )
#                                            )
#             )
#         year_select = driver.find_element(By.CLASS_NAME,
#                                           'select2-search__field'
#                                           )
#         year_select.send_keys(year)
#         year_select.send_keys(Keys.RETURN)

#         # Тоже самое для поля с выбором категории транспорта
#         element = WebDriverWait(
#             driver,
#             10
#             ).until(
#                 EC.presence_of_element_located((
#                     By.ID,
#                     "select2-ctl00_ctl00_ctl00_ddlTaxCategory-container"
#                     )
#                                                )
#                 )
#         cat_select_field = driver.find_element(
#             By.ID,
#             "select2-ctl00_ctl00_ctl00_ddlTaxCategory-container"
#             )
#         cat_select = WebDriverWait(driver, 20).until(
#             EC.element_to_be_clickable(cat_select_field)
#             )
#         cat_select.click()
#         element = WebDriverWait(driver, 10).until(
#             EC.presence_of_element_located((By.CLASS_NAME,
#                                             'select2-search__field'
#                                             )
#                                            )
#             )
#         cat_select = driver.find_element(By.CLASS_NAME,
#                                          'select2-search__field'
#                                          )

#         # Для Новосибирска форма ввода данных меняется, появляются разделения
#         # на группы по мощности и году выпуска
#         if data.loc[ind, 'file_name'] == 'novsib.csv':

#             # Автомобили легковые с мощностью двигателя до 100 л.с.
#             # включительно
#             if data.loc[ind, 'eng_hpower'] < 100:
#                 n = 1

#             # Автомобили легковые с мощностью двигателя свыше 100 л.с. до
#             # 150 л.с. включительно
#             elif 100 < data.loc[ind, 'eng_hpower'] <= 150:
#                 n = 2

#             # Автомобили легковые с мощностью двигателя свыше 150 л.с. до
#             # 200 л.с. включительно
#             elif 150 < data.loc[ind, 'eng_hpower'] <= 200:

#                 # с учетом количества лет, прошедших с года выпуска
#                 # транспортного средства, до 5 лет включительно
#                 if int(year) - int(data.loc[ind, 'year']) <= 5:
#                     n = 3

#                 # с учетом количества лет, прошедших с года выпуска
#                 # транспортного средства, от 5 лет до 10 лет включительно
#                 elif 5 < int(year) - int(data.loc[ind, 'year']) <= 10:
#                     n = 4

#                 # с учетом количества лет, прошедших с года выпуска
#                 # транспортного средства, свыше 10 лет
#                 elif int(year) - int(data.loc[ind, 'year']) > 10:
#                     n = 5

#             # Автомобили легковые с мощностью двигателя свыше 200 л.с.
#             # до 250 л.с. включительно
#             elif 200 < data.loc[ind, 'eng_hpower'] <= 250:

#                 # с учетом количества лет, прошедших с года выпуска
#                 # транспортного средства, до 5 лет включительно
#                 if int(year) - int(data.loc[ind, 'year']) <= 5:
#                     n = 6

#                 # с учетом количества лет, прошедших с года выпуска
#                 # транспортного средства, от 5 лет до 10 лет включительно
#                 # так как отсутствует группа свыше 10 лет, будем считать,
#                 # что она входит в эту группу
#                 else:
#                     n = 7

#             # Автомобили легковые с мощностью двигателя свыше 250 л.с.
#             else:

#                 # с учетом количества лет, прошедших с года выпуска
#                 # транспортного средства, до 5 лет включительно
#                 if int(year) - int(data.loc[ind, 'year']) <= 5:
#                     n = 8
#                 # с учетом количества лет, прошедших с года выпуска
#                 # транспортного средства, от 5 лет до 10 лет включительно
#                 elif 5 < int(year) - int(data.loc[ind, 'year']) <= 10:
#                     n = 9
#                 # с учетом количества лет, прошедших с года выпуска
#                 # транспортного средства, свыше 10 лет
#                 elif int(year) - int(data.loc[ind, 'year']) > 10:
#                     n = 10

#             # Находим поле с вариантами категорий
#             fild_cat = driver.find_element(
#                 By.XPATH,
#                 '//*[@id="select2-ctl00_ctl00_ctl00_ddlTaxCategory-results"]'
#                 )

#             # Создаем экземпляр класса ActionChains
#             actions = ActionChains(driver)

#             # Перемещаемся с полю с вариантами категорий
#             actions.move_to_element(fild_cat).perform()

#             # В цикле передвигаемся по вариантам до нужного значения n нажимая
#             # стрелку вниз
#             for i in range(n-1):

#                 # Прокрутка вниз
#                 actions.send_keys(Keys.ARROW_DOWN).perform()

#             # Кликаем по текущему элементу в поле выбора вариантов категорий
#             actions.send_keys(Keys.ENTER).perform()

#         # Для других городов вбиваем часть наименования категории легковых
#         # автомобилейи жмем ввод
#         else:
#             cat_select.send_keys('легко')
#             cat_select.send_keys(Keys.RETURN)

#         # Тоже самое для поля с указанием мощности автомобиля
#         element = WebDriverWait(driver, 10).until(
#             EC.presence_of_element_located((By.ID,
#                                             "ctl00_ctl00_ctl00_tbValue"
#                                             )
#                                            )
#             )
#         power_select = driver.find_element(By.ID, "ctl00_ctl00_ctl00_tbValue")
#         power_select.send_keys(str(data.loc[ind, 'eng_hpower']))
#         power_select.send_keys(Keys.RETURN)

#         # В ходе работы выяснилось, что иногда появляется поле с указанием
#         # года выпуска автомобиля. На всякий случай задаем небольшую задержку
#         time.sleep(2)

#         # и пытаемся повторить процедуру для поля с годом выпуска
#         try:
#             element = WebDriverWait(driver, 10).until(
#                 EC.presence_of_element_located((By.ID,
#                                                 "ctl00_ctl00_ctl00_tbRateYear"
#                                                 )
#                                                )
#                 )
#             rate_y = driver.find_element(By.ID,
#                                          "ctl00_ctl00_ctl00_tbRateYear"
#                                          )
#             rate_y.send_keys(str(data.loc[ind, 'year']))
#             rate_y.send_keys(Keys.RETURN)

#         # Если поле не появляется - продолжаем работу
#         except:
#             pass

#         # Немного прокручиваем окно вниз так как интересующая нас кнопка
#         # "рассчитать" может не подгрузиться
#         driver. execute_script("window. scrollBy(0, 300)")

#         # Ожидаем появления кнопки, находим ее и кликаем по ней
#         element = WebDriverWait(driver, 10).until(
#             EC.presence_of_element_located((
#                 By.XPATH,
#                 '//*[@id="ctl00_ctl00_ctl00_b_Go"]'
#                 )
#                                            )
#             )
#         b_button = driver.find_element(By.XPATH,
#                                        '//*[@id="ctl00_ctl00_ctl00_b_Go"]'
#                                        )
#         button_ok = WebDriverWait(driver, 10).until(
#             EC.element_to_be_clickable(b_button)
#             )
#         button_ok.click()

#         # Еще немного прокручиваем окно вниз
#         driver. execute_script("window. scrollBy(0, 300)")

#         # Иногда вместо кнопки расчета появляется кнопка "далее", если это так
#         # то мы уже нажали на неё в блоке выше, а здесь пытаемся снова найти
#         # кнопку для рассчета и нажать на неё
#         try:
#             time.sleep(1)
#             b_button2 = driver.find_element(
#                 By.XPATH,
#                 '//*[@id="ctl00_ctl00_ctl00_b_Go"]'
#                 )
#             button_ok = WebDriverWait(driver, 10).until(
#                 EC.element_to_be_clickable(b_button2)
#                 )
#             button_ok.click()
#         except:
#             pass

#         # Ждем появление результатов расчета и находим их
#         element = WebDriverWait(
#             driver,
#             10
#             ).until(EC.presence_of_element_located((
#                 By.XPATH,
#                 '//*[@id="ctl00_ctl00_ctl00_dResultText"]/div[5]/div[2]/strong'
#                 )
#                                                    )
#                     )
#         tax = driver.find_element(
#             By.XPATH,
#             '//*[@id="ctl00_ctl00_ctl00_dResultText"]/div[5]/div[2]/strong'
#             )

#         with open('./data/rest_tax.csv', 'a', encoding='utf-8') as f:
#             f.write(str(ind) + ';' + tax.text + '\n')

#     except:
#         tax = "None"

#     # Записываем в файл текущий индекс и полученное налога
#         with open('./data/rest_tax.csv', 'a', encoding='utf-8') as f:
#             f.write(str(ind) + ';' + tax + '\n')

#     # Ждем появления кнопки "произвести новый расчет", находим и нажимаем
#     # на неё
#     element = WebDriverWait(driver, 10).until(
#         EC.presence_of_element_located((
#             By.CLASS_NAME,
#             "button.button_border.noactive "
#             )
#                                        )
#         )
#     next_find = driver.find_element(By.CLASS_NAME,
#                                     "button.button_border.noactive "
#                                     )
#     button_ok = WebDriverWait(driver, 10).until(
#         EC.element_to_be_clickable(next_find)
#         )
#     button_ok.click()

# # закрываем окно и выходим из вебдрайвера
# driver.close()
# driver.quit()

# print('-------------complite----------------')


Количество пропусков в призенаке tax: 1654


In [ ]:
# Загружаем восстановленные данные по транспортному налогу, полученные в
# ячейке выше
df = pd.read_csv('./data/rest_tax.csv',
                 sep=';',
                 index_col=0,
                 names=['tax_restore']
                 )

# Объединяем с нашим датасетом
data = data.join(df)

# Проводим подстановку отсутствующих значений по транспортному налогу
data['tax'] = data.apply(lambda x: x['tax_restore'] if
                         x['tax'] == 'None' else x['tax'], axis=1)

# Убираем вспомогательный столбец 'tax_restore' из данных
data = data.drop(['tax_restore'], axis=1)

# В востановленных данных формат вывода с копейками после запятой.
# Избавимся от дробной части - она везде равна 0
data['tax'] = data['tax'].map(lambda x: x.split(',')[0])

# Сразу избавимся от размерности в значениях (р/год)
data['tax'] = data['tax'].map(lambda x: ''.join([i for i in x if i.isdigit()]))

# Проверяем результат
print(f"Количество пропусков: {data['tax'].isna().sum()}")


Количество пропусков: 0


In [ ]:
# Получим номера объявлений из поля head с помощью функции get_id из
# модуля myfunc
data['id'] = data['head'].map(get_id)

# Проверим наличие дубликатов по номеру объявления
print(f"Количество дубликатов до очистки: {data['id'].duplicated().sum()}")

# Избавимся от дубликатов
data.drop_duplicates(inplace=True, subset=['id'])
print(f"Количество дубликатов после очистки: {data['id'].duplicated().sum()}")

# Переназначим индексы
data.reset_index(drop=True, inplace=True)


Количество дубликатов до очистки: 4039
Количество дубликатов после очистки: 0


In [ ]:
# Получим координаты городов, где было размещено объявление.
# Зададим пустой словарь для сбора координат городов
coordinate = {}

# Используем геокодер Nominatim.
# Для работы нужно указать название приложения.
geolocator = Nominatim(user_agent="Tester")

# В цикле перебираем уникальные наименования городов
for elem in data['sity'].unique():

    # Передаем наименование города в метод geocode экземпляра геолокатора.
    # На выходе получаем полный адрес и кортеж с координатами. Записываем
    # в переменную координаты объекта.
    loc = geolocator.geocode(elem)[-1]

    # Вносим координаты в словарь для текущего наименования города
    coordinate[elem] = loc

# Добавляем признаки с координатами по наименованию города
data['lat'] = data['sity'].map(lambda x: coordinate[x][0])
data['long'] = data['sity'].map(lambda x: coordinate[x][1])


In [ ]:
# Проверим процент пропусков в наших признаках. Признаки с пропусками
# более 30 % использовать не будем. В цикле перебираем все наименования
# признаков
for name in data.columns:
    # Вычисляем процент пропусков в текущем признаке
    skip = data[data[name] == 'None'].shape[0]/data.shape[0]*100

    # и если он более 0 выводим на экран
    if skip > 0:
        print(f"Процент пропусков в признаке {name}: "
              f"{skip:.2f} %")


Процент пропусков в признаке availability: 2.64 %
Процент пропусков в признаке generation: 9.91 %
Процент пропусков в признаке equipment: 3.54 %
Процент пропусков в признаке steering_wheel: 2.64 %
Процент пропусков в признаке condition: 2.64 %
Процент пропусков в признаке owners: 4.42 %
Процент пропусков в признаке pts: 4.62 %
Процент пропусков в признаке possession: 83.09 %
Процент пропусков в признаке exchange: 63.10 %
Процент пропусков в признаке vin: 36.77 %
Процент пропусков в признаке state_number: 70.04 %


In [ ]:
# В некоторых признаках (наличие, поколение, состояние, кол-во владельцев,
# ПТС) вполне можно заменить пропуски значением 'Не указано'. Создадим
# список с этими признаками
features = ['availability', 'generation', 'condition', 'owners', 'pts']

# В цикле перебираем признаки из списка
for elem in features:

    # Для каждого признака проводим замену пропуска значением 'Не указано'
    data[elem] = data[elem].map(lambda x: 'Не указано' if x == 'None' else x)


In [ ]:
# В признаке тип руля (левый/правый) попробуем восстановить пропущенные
# значения
print(f"Количество пропусков в признаке до заполнения: "
      f"{len(list(data[data['steering_wheel'] == 'None'].index))}")

# Получим список индексов для этих пропусков
ind_ls = list(data[data['steering_wheel'] == 'None'].index)

# Создаем словарь для сбора информации
wheel_dict = {}

# В цикле перебираем индексы
for ind in ind_ls:

    # На каждой итерации получаем значение марки, модели и поколения автомобиля
    brand = data.loc[ind, 'brand']
    model = data.loc[ind, 'model']
    gen = data.loc[ind, 'generation']

    # Фильтруем исходный датасет по этим трем значениям
    elem = data[(data['brand'] == brand) &
                (data['model'] == model) &
                (data['generation'] == gen)]['steering_wheel']

    # Записываем в переменную наиболее популярное значение
    wheel = elem.value_counts().index[0]

    # Если наиболее популярное значение оказалось 'None':
    if wheel == 'None':

        # Возможно у нас было всего одно совпадение. Пробуем взять следующее
        # по популярности
        try:
            wheel = elem.value_counts().index[1]

        # Если его не оказалось - значит совпадений не нашлось. Оставляем
        # текущее значение.
        except:
            continue

    # Заносим в словарь по ключу - индексу значение типа руля
    wheel_dict[ind] = wheel

# Создаем вспомогательный датасет с восстановленными значениями и
# соответствующими им индексами
df = pd.DataFrame(wheel_dict.items(), columns=['ind', 'rest_w'])
df = df.set_index('ind')

# Объединяем датасеты
data = data.join(df)

# Заполняем пропуски в признаке 'steering_wheel'
data['steering_wheel'] = data.apply(lambda x: x['rest_w'] if
                                    x['steering_wheel'] == 'None'
                                    else x['steering_wheel'], axis=1)

# Удаляем вспомогательный столбец
data = data.drop(['rest_w'], axis=1)

# Проверяем количество пропусков
print(f"Количество пропусков в признаке после 1ой итерации заполнения: "
      f"{len(list(data[data['steering_wheel'].isna()].index))}")


Количество пропусков в признаке до заполнения: 946
Количество пропусков в признаке после 1ой итерации заполнения: 30


In [ ]:
# Для заполнения оставшихся пропусков повторим ту же процедуру, но без
# поколения автомобиля
ind_ls = list(data[data['steering_wheel'].isna()].index)

# Создаем словарь для сбора информации
wheel_dict = {}

# В цикле перебираем индексы
for ind in ind_ls:

    # На каждой итерации получаем значение марки и модели автомобиля
    brand = data.loc[ind, 'brand']
    model = data.loc[ind, 'model']

    # Фильтруем исходный датасет по этим двум значениям
    elem = data[(data['brand'] == brand) &
                (data['model'] == model)]['steering_wheel']

    # Пытаемся записать в переменную наиболее популярное значение
    try:
        wheel = elem.value_counts().index[0]

    # Если его нет заносим в переменную 'None'
    except:
        wheel = 'None'

    # Заносим в словарь по ключу - индексу значение типа руля
    wheel_dict[ind] = wheel

# Создаем вспомогательный датасет с восстановленными значениями и
# соответствующими им индексами
df = pd.DataFrame(wheel_dict.items(), columns=['ind', 'rest_w'])
df = df.set_index('ind')

# Объединяем датасеты
data = data.join(df)

# Заполняем пропуски в признаке 'steering_wheel'
data['steering_wheel'] = data.apply(lambda x: x['rest_w'] if
                                    pd.isnull(x['steering_wheel'])
                                    else x['steering_wheel'], axis=1)

# Удаляем вспомогательный столбец
data = data.drop(['rest_w'], axis=1)

# Проверяем количество пропусков
print(f"Количество пропусков в признаке после 2ой итерации заполнения: "
      f"{len(list(data[data['steering_wheel']=='None'].index))}")


Количество пропусков в признаке после 2ой итерации заполнения: 21


In [ ]:
# Оставшиеся пропуски заполним модой
# Создаем словарь для хранения значений
wheel_dict = {}

# В цикле перебираем марки автомобилей с пропущенным значением
for brand in list(data[data['steering_wheel'] == 'None']['brand'].unique()):

    # Для каждой марки определяем моду по признаку типа руля
    mode = data[data['brand'] == brand]['steering_wheel'].mode().values[0]

    # Заносим значение моды в словарь по ключу - марке автомобиля
    wheel_dict[brand] = mode

# Заполняем пропуски, используя лямбда функцию
data['steering_wheel'] = data.apply(lambda x: wheel_dict[x['brand']] if
                                    x['steering_wheel'] == 'None'
                                    else x['steering_wheel'], axis=1)

# Проверяем количество пропусков
print(f"Количество пропусков в признаке после 3ей итерации заполнения: "
      f"{len(list(data[data['steering_wheel']=='None'].index))}")


Количество пропусков в признаке после 3ей итерации заполнения: 0


In [ ]:
# Поле  "options" содержит информацию о комплектации/опциях автомобиля.
# Опции разделены на 8 категорий. Создаем вспомогательный датасет применив
# нашу функцию get_cat_options из модуля myfunc
df = pd.DataFrame(data['options'].apply(get_cat_options).to_list(),
                  columns=['option_media',
                           'option_interior',
                           'option_exterior',
                           'option_visibility',
                           'option_safety',
                           'option_comfort',
                           'option_protection',
                           'option_other'
                           ]
                  )

# Проверяем результат
display(df.head())


,option_media,option_interior,option_exterior,option_visibility,option_safety,option_comfort,option_protection,option_other
0,"[Мультимедиа система с ЖК-экраном, Аудиосистем...","[Искусственная кожа (материал салона), Регулир...",[Диски 17],"[Датчик света, Дневные ходовые огни, Светодиод...","[Подушка безопасности водителя, Подушка безопа...","[Климат-контроль 1-зонный, Кондиционер, Круиз-...","[Центральный замок, Иммобилайзер]",Не указано
1,Не указано,[Количество мест: 5],Не указано,Не указано,Не указано,Не указано,Не указано,Не указано
2,Не указано,[Количество мест: 4],Не указано,Не указано,Не указано,Не указано,Не указано,Не указано
3,"[AUX, Розетка 12V, Аудиосистема, Аудиоподготовка]","[Кожа (материал салона), Спортивные передние с...",[Диски 19],"[Противотуманные фары, Датчик света, Автоматич...","[Подушка безопасности водителя, Подушка безопа...","[Климат-контроль 1-зонный, Кондиционер, Регули...","[Сигнализация, Центральный замок, Датчик прони...",Не указано
4,"[AUX, Розетка 12V, Аудиосистема, Аудиоподготовка]","[Ткань (материал салона), Подогрев передних си...","[Диски 17, Декоративные молдинги]","[Противотуманные фары, Датчик дождя, Автоматич...","[Подушка безопасности водителя, Подушка безопа...","[Климат-контроль 1-зонный, Кондиционер, Регули...","[Центральный замок, Иммобилайзер]",Не указано


In [ ]:
# Сами опции также имеют фиксированные позиции и их можно разбить на отдельные
# признаки. Начнем с категории безопасность - сформируем датасет с опциями из
# категории "безопасность" применив функцию get_safety из модуля myfunc
df_option = pd.DataFrame(df['option_safety'].apply(get_safety).to_list(),
                         columns=['airbags_driv',
                                  'airbags_pass',
                                  'airbags_side',
                                  'airbags_s_rear',
                                  'airbags_knees_driv',
                                  'airbags_knees_pass',
                                  'airbags_wind',
                                  'collision_warn',
                                  'collision_avoid',
                                  'lane_depart_warn',
                                  'lane_holds',
                                  'traffic_jam_assist',
                                  'driv_fatigue_sens',
                                  'road_sign_recogn',
                                  'asr',
                                  'vsm',
                                  'bas_ebd',
                                  'hill_start_assist',
                                  'descent_assist',
                                  'blind_spot_monitor',
                                  'assist_revers_parking',
                                  'night_vision',
                                  'isofix_rear',
                                  'isofix_front',
                                  'pressure_sensor',
                                  'abs_sys',
                                  'esp',
                                  'lockout',
                                  'era_glonass',
                                  'armored_body'
                                  ]
                         )

# Избавимся от лишнего символа '•'
for col in df_option.columns:
    df_option[col] = df_option[col].map(lambda x: ''.join(x.split('•')))

# Добавляем полученный результат к вспомогательному датасету
df = pd.concat([df, df_option], axis=1)

# Проверяем результат
display(df.head())


,option_media,option_interior,option_exterior,option_visibility,option_safety,option_comfort,option_protection,option_other,airbags_driv,airbags_pass,...,assist_revers_parking,night_vision,isofix_rear,isofix_front,pressure_sensor,abs_sys,esp,lockout,era_glonass,armored_body
0,"[Мультимедиа система с ЖК-экраном, Аудиосистем...","[Искусственная кожа (материал салона), Регулир...",[Диски 17],"[Датчик света, Дневные ходовые огни, Светодиод...","[Подушка безопасности водителя, Подушка безопа...","[Климат-контроль 1-зонный, Кондиционер, Круиз-...","[Центральный замок, Иммобилайзер]",Не указано,Подушка безопасности водителя,Подушка безопасности пассажира,...,Не указано,Не указано,Крепление детского кресла (задний ряд) ISOFIX,Не указано,Датчик давления в шинах,Антиблокировочная система (ABS),Система стабилизации (ESP),Не указано,Не указано,Не указано
1,Не указано,[Количество мест: 5],Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,...,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано
2,Не указано,[Количество мест: 4],Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,...,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано
3,"[AUX, Розетка 12V, Аудиосистема, Аудиоподготовка]","[Кожа (материал салона), Спортивные передние с...",[Диски 19],"[Противотуманные фары, Датчик света, Автоматич...","[Подушка безопасности водителя, Подушка безопа...","[Климат-контроль 1-зонный, Кондиционер, Регули...","[Сигнализация, Центральный замок, Датчик прони...",Не указано,Подушка безопасности водителя,Подушка безопасности пассажира,...,Не указано,Не указано,Не указано,Не указано,Датчик давления в шинах,Антиблокировочная система (ABS),Система стабилизации (ESP),Не указано,Не указано,Не указано
4,"[AUX, Розетка 12V, Аудиосистема, Аудиоподготовка]","[Ткань (материал салона), Подогрев передних си...","[Диски 17, Декоративные молдинги]","[Противотуманные фары, Датчик дождя, Автоматич...","[Подушка безопасности водителя, Подушка безопа...","[Климат-контроль 1-зонный, Кондиционер, Регули...","[Центральный замок, Иммобилайзер]",Не указано,Подушка безопасности водителя,Подушка безопасности пассажира,...,Не указано,Не указано,Крепление детского кресла (задний ряд) ISOFIX,Не указано,Датчик давления в шинах,Антиблокировочная система (ABS),Система стабилизации (ESP),Не указано,Не указано,Не указано


In [ ]:
# Сформируем датасет с опциями из категории "обзор" применив функцию
# get_visibility из модуля myfunc
df_option = pd.DataFrame(df['option_visibility'].apply(get_visibility)
                         .to_list(), columns=['laser_headlights',
                                              'led_headlights',
                                              'xenon_headlights',
                                              'heat_wind_wipers',
                                              'heat_wind',
                                              'heat_mirrors',
                                              'heat_wind_washer',
                                              'anti_frog',
                                              'adaptive_light',
                                              'rain_sens',
                                              'light_sens',
                                              'day_lights',
                                              'auto_headlight_corr',
                                              'auto_high_beam',
                                              'headlight_washer'
                                              ]
                         )

# Избавимся от лишнего символа '•'
for col in df_option.columns:
    df_option[col] = df_option[col].map(lambda x: ''.join(x.split('•')))

# Добавляем полученный результат к вспомогательному датасету
df = pd.concat([df, df_option], axis=1)

# Проверяем результат
display(df.head())


,option_media,option_interior,option_exterior,option_visibility,option_safety,option_comfort,option_protection,option_other,airbags_driv,airbags_pass,...,heat_mirrors,heat_wind_washer,anti_frog,adaptive_light,rain_sens,light_sens,day_lights,auto_headlight_corr,auto_high_beam,headlight_washer
0,"[Мультимедиа система с ЖК-экраном, Аудиосистем...","[Искусственная кожа (материал салона), Регулир...",[Диски 17],"[Датчик света, Дневные ходовые огни, Светодиод...","[Подушка безопасности водителя, Подушка безопа...","[Климат-контроль 1-зонный, Кондиционер, Круиз-...","[Центральный замок, Иммобилайзер]",Не указано,Подушка безопасности водителя,Подушка безопасности пассажира,...,Электрообогрев боковых зеркал,Не указано,Не указано,Не указано,Не указано,Датчик света,Дневные ходовые огни,Не указано,Не указано,Не указано
1,Не указано,[Количество мест: 5],Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,...,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано
2,Не указано,[Количество мест: 4],Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,...,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано
3,"[AUX, Розетка 12V, Аудиосистема, Аудиоподготовка]","[Кожа (материал салона), Спортивные передние с...",[Диски 19],"[Противотуманные фары, Датчик света, Автоматич...","[Подушка безопасности водителя, Подушка безопа...","[Климат-контроль 1-зонный, Кондиционер, Регули...","[Сигнализация, Центральный замок, Датчик прони...",Не указано,Подушка безопасности водителя,Подушка безопасности пассажира,...,Электрообогрев боковых зеркал,Электрообогрев форсунок стеклоомывателей,Противотуманные фары,Не указано,Не указано,Датчик света,Не указано,Автоматический корректор фар,Не указано,Омыватель фар
4,"[AUX, Розетка 12V, Аудиосистема, Аудиоподготовка]","[Ткань (материал салона), Подогрев передних си...","[Диски 17, Декоративные молдинги]","[Противотуманные фары, Датчик дождя, Автоматич...","[Подушка безопасности водителя, Подушка безопа...","[Климат-контроль 1-зонный, Кондиционер, Регули...","[Центральный замок, Иммобилайзер]",Не указано,Подушка безопасности водителя,Подушка безопасности пассажира,...,Электрообогрев боковых зеркал,Электрообогрев форсунок стеклоомывателей,Противотуманные фары,Не указано,Датчик дождя,Не указано,Не указано,Автоматический корректор фар,Не указано,Омыватель фар


In [ ]:
# Сформируем датасет с опциями из категории "мультимедиа" применив функцию
# get_media из модуля myfunc
df_option = pd.DataFrame(df['option_media'].apply(get_media).to_list(),
                         columns=['premium_audio',
                                  'audio',
                                  'audio_prep',
                                  'ru_menu',
                                  'lcd_screen',
                                  'media_rear_pass',
                                  'remote_control',
                                  'wireless_charg',
                                  'usb',
                                  'navi_sys',
                                  'voice_control',
                                  'android_auto',
                                  'carplay',
                                  'yandex_auto',
                                  'aux',
                                  'bluetooth',
                                  'socket_12v',
                                  'socket_220v'
                                  ]
                         )

# Избавимся от лишнего символа '•'
for col in df_option.columns:
    df_option[col] = df_option[col].map(lambda x: ''.join(x.split('•')))

# Добавляем полученный результат к вспомогательному датасету
df = pd.concat([df, df_option], axis=1)

# Проверяем результат
display(df.head())


,option_media,option_interior,option_exterior,option_visibility,option_safety,option_comfort,option_protection,option_other,airbags_driv,airbags_pass,...,usb,navi_sys,voice_control,android_auto,carplay,yandex_auto,aux,bluetooth,socket_12v,socket_220v
0,"[Мультимедиа система с ЖК-экраном, Аудиосистем...","[Искусственная кожа (материал салона), Регулир...",[Диски 17],"[Датчик света, Дневные ходовые огни, Светодиод...","[Подушка безопасности водителя, Подушка безопа...","[Климат-контроль 1-зонный, Кондиционер, Круиз-...","[Центральный замок, Иммобилайзер]",Не указано,Подушка безопасности водителя,Подушка безопасности пассажира,...,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано
1,Не указано,[Количество мест: 5],Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,...,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано
2,Не указано,[Количество мест: 4],Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,...,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано
3,"[AUX, Розетка 12V, Аудиосистема, Аудиоподготовка]","[Кожа (материал салона), Спортивные передние с...",[Диски 19],"[Противотуманные фары, Датчик света, Автоматич...","[Подушка безопасности водителя, Подушка безопа...","[Климат-контроль 1-зонный, Кондиционер, Регули...","[Сигнализация, Центральный замок, Датчик прони...",Не указано,Подушка безопасности водителя,Подушка безопасности пассажира,...,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,AUX,Не указано,Розетка 12V,Не указано
4,"[AUX, Розетка 12V, Аудиосистема, Аудиоподготовка]","[Ткань (материал салона), Подогрев передних си...","[Диски 17, Декоративные молдинги]","[Противотуманные фары, Датчик дождя, Автоматич...","[Подушка безопасности водителя, Подушка безопа...","[Климат-контроль 1-зонный, Кондиционер, Регули...","[Центральный замок, Иммобилайзер]",Не указано,Подушка безопасности водителя,Подушка безопасности пассажира,...,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,AUX,Не указано,Розетка 12V,Не указано


In [ ]:
# Сформируем датасет с опциями из категории "комфорт" применив нашу функцию
# get_comfort из модуля myfunc
df_option = pd.DataFrame(df['option_comfort'].apply(get_comfort).to_list(),
                         columns=['climate_1_zone',
                                  'climate_2_zone',
                                  'climate_m_zone',
                                  'air_cond',
                                  'cam_360',
                                  'cam_front',
                                  'cam_rear',
                                  'e_wind_font',
                                  'e_wind_rear',
                                  'cruise_control',
                                  'a_cruise_control',
                                  's_wheel_adj_height',
                                  's_wheel_adj_reach',
                                  's_wheel_adj_elect',
                                  's_wheel_adj_mem',
                                  'amplifier_wheel',
                                  'a_amplifier_wheel',
                                  'front_park',
                                  'rear_park',
                                  'auto_park',
                                  'head_up_disp',
                                  'drive_mode',
                                  'remote_start',
                                  'trunk_without_hands',
                                  'multifunc_wheel',
                                  'on_board_comp',
                                  'electr_instr_panel',
                                  'keyless_entry_sys',
                                  'engine_button',
                                  'prog_pre_heater',
                                  'electric_trunk',
                                  'electr_fold_mirror',
                                  'mirror_memory',
                                  'paddle_shifters',
                                  'pedal_assembly',
                                  'start_stop_sys',
                                  'electric_mirror',
                                  'glove_box',
                                  'door_closer',
                                  'lighter_ashtray'
                                  ]
                         )

# Избавимся от лишнего символа '•'
for col in df_option.columns:
    df_option[col] = df_option[col].map(lambda x: ''.join(x.split('•')))

# Добавляем полученный результат к вспомогательному датасету
df = pd.concat([df, df_option], axis=1)

# Проверяем результат
display(df.head())


,option_media,option_interior,option_exterior,option_visibility,option_safety,option_comfort,option_protection,option_other,airbags_driv,airbags_pass,...,electric_trunk,electr_fold_mirror,mirror_memory,paddle_shifters,pedal_assembly,start_stop_sys,electric_mirror,glove_box,door_closer,lighter_ashtray
0,"[Мультимедиа система с ЖК-экраном, Аудиосистем...","[Искусственная кожа (материал салона), Регулир...",[Диски 17],"[Датчик света, Дневные ходовые огни, Светодиод...","[Подушка безопасности водителя, Подушка безопа...","[Климат-контроль 1-зонный, Кондиционер, Круиз-...","[Центральный замок, Иммобилайзер]",Не указано,Подушка безопасности водителя,Подушка безопасности пассажира,...,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Электропривод зеркал,Не указано,Не указано,Не указано
1,Не указано,[Количество мест: 5],Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,...,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано
2,Не указано,[Количество мест: 4],Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,...,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано
3,"[AUX, Розетка 12V, Аудиосистема, Аудиоподготовка]","[Кожа (материал салона), Спортивные передние с...",[Диски 19],"[Противотуманные фары, Датчик света, Автоматич...","[Подушка безопасности водителя, Подушка безопа...","[Климат-контроль 1-зонный, Кондиционер, Регули...","[Сигнализация, Центральный замок, Датчик прони...",Не указано,Подушка безопасности водителя,Подушка безопасности пассажира,...,Не указано,Не указано,Не указано,Не указано,Не указано,Система «старт-стоп»,Электропривод зеркал,Не указано,Не указано,Прикуриватель и пепельница
4,"[AUX, Розетка 12V, Аудиосистема, Аудиоподготовка]","[Ткань (материал салона), Подогрев передних си...","[Диски 17, Декоративные молдинги]","[Противотуманные фары, Датчик дождя, Автоматич...","[Подушка безопасности водителя, Подушка безопа...","[Климат-контроль 1-зонный, Кондиционер, Регули...","[Центральный замок, Иммобилайзер]",Не указано,Подушка безопасности водителя,Подушка безопасности пассажира,...,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Электропривод зеркал,Охлаждаемый перчаточный ящик,Не указано,Прикуриватель и пепельница


In [ ]:
# Сформируем датасет с опциями из категории "экстерьер" применив функцию
# get_exterior из модуля myfunc
df_option = pd.DataFrame(df['option_exterior'].apply(get_exterior).to_list(),
                         columns=['d_type_steel',
                                  'd_type_light',
                                  'd_size_12',
                                  'd_size_13',
                                  'd_size_14',
                                  'd_size_15',
                                  'd_size_16',
                                  'd_size_17',
                                  'd_size_18',
                                  'd_size_19',
                                  'd_size_20',
                                  'd_size_21',
                                  "d_size_22",
                                  'd_size_23',
                                  'd_size_24',
                                  'd_size_25',
                                  'd_size_26',
                                  'd_size_27',
                                  'd_size_28',
                                  'two_paint',
                                  'body_kit',
                                  'roof_rails',
                                  'airbrush',
                                  'moldings'
                                  ]
                         )

# Избавимся от лишнего символа '•'
for col in df_option.columns:
    df_option[col] = df_option[col].map(lambda x: ''.join(x.split('•')))

# В цикле переберем все возможные размеры дисков и преобразуем их в число
for i in range(12, 29):
    df_option[f'd_size_{i}'] = df_option[f'd_size_{i}'].map(
        lambda x: int(x.split(' ')[-1]) if x != 'Не указано' else 0
        )

# В некоторых объявлениях указаны диски разных размеров, создадим новый
# признак, указав в нем максимальный размер дисков
df_option['d_size'] = df_option[['d_size_12',
                                 'd_size_13',
                                 'd_size_14',
                                 'd_size_15',
                                 'd_size_16',
                                 'd_size_17',
                                 'd_size_18',
                                 'd_size_19',
                                 'd_size_20',
                                 'd_size_21',
                                 'd_size_22',
                                 'd_size_23',
                                 'd_size_24',
                                 'd_size_25',
                                 'd_size_26',
                                 'd_size_27',
                                 'd_size_28'
                                 ]
                                ].max(axis=1)
# Удалим признаки по каждому размеру диска
df_option.drop(['d_size_12',
                'd_size_13',
                'd_size_14',
                'd_size_15',
                'd_size_16',
                'd_size_17',
                'd_size_18',
                'd_size_19',
                'd_size_20',
                'd_size_21',
                'd_size_22',
                'd_size_23',
                'd_size_24',
                'd_size_25',
                'd_size_26',
                'd_size_27',
                'd_size_28'
                ],
               axis=1,
               inplace=True
               )

# Добавляем полученный результат к вспомогательному датасету
df = pd.concat([df, df_option], axis=1)

# Проверяем результат
display(df.head())


,option_media,option_interior,option_exterior,option_visibility,option_safety,option_comfort,option_protection,option_other,airbags_driv,airbags_pass,...,door_closer,lighter_ashtray,d_type_steel,d_type_light,two_paint,body_kit,roof_rails,airbrush,moldings,d_size
0,"[Мультимедиа система с ЖК-экраном, Аудиосистем...","[Искусственная кожа (материал салона), Регулир...",[Диски 17],"[Датчик света, Дневные ходовые огни, Светодиод...","[Подушка безопасности водителя, Подушка безопа...","[Климат-контроль 1-зонный, Кондиционер, Круиз-...","[Центральный замок, Иммобилайзер]",Не указано,Подушка безопасности водителя,Подушка безопасности пассажира,...,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,17
1,Не указано,[Количество мест: 5],Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,...,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,0
2,Не указано,[Количество мест: 4],Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,...,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,0
3,"[AUX, Розетка 12V, Аудиосистема, Аудиоподготовка]","[Кожа (материал салона), Спортивные передние с...",[Диски 19],"[Противотуманные фары, Датчик света, Автоматич...","[Подушка безопасности водителя, Подушка безопа...","[Климат-контроль 1-зонный, Кондиционер, Регули...","[Сигнализация, Центральный замок, Датчик прони...",Не указано,Подушка безопасности водителя,Подушка безопасности пассажира,...,Не указано,Прикуриватель и пепельница,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,19
4,"[AUX, Розетка 12V, Аудиосистема, Аудиоподготовка]","[Ткань (материал салона), Подогрев передних си...","[Диски 17, Декоративные молдинги]","[Противотуманные фары, Датчик дождя, Автоматич...","[Подушка безопасности водителя, Подушка безопа...","[Климат-контроль 1-зонный, Кондиционер, Регули...","[Центральный замок, Иммобилайзер]",Не указано,Подушка безопасности водителя,Подушка безопасности пассажира,...,Не указано,Прикуриватель и пепельница,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Декоративные молдинги,17


In [ ]:
# Сформируем датасет с опциями из категории "защита от угона" применив функцию
# get_protection из модуля myfunc
df_option = pd.DataFrame(df['option_protection'].apply(get_protection)
                         .to_list(), columns=['signaling',
                                              'fback_signaling',
                                              'central_lock',
                                              'vol_sens',
                                              'immobilizer'
                                              ]
                         )

# Избавимся от лишнего символа '•'
for col in df_option.columns:
    df_option[col] = df_option[col].map(lambda x: ''.join(x.split('•')))

# Добавляем полученный результат к вспомогательному датасету
df = pd.concat([df, df_option], axis=1)

# Проверяем результат
display(df.head())


,option_media,option_interior,option_exterior,option_visibility,option_safety,option_comfort,option_protection,option_other,airbags_driv,airbags_pass,...,body_kit,roof_rails,airbrush,moldings,d_size,signaling,fback_signaling,central_lock,vol_sens,immobilizer
0,"[Мультимедиа система с ЖК-экраном, Аудиосистем...","[Искусственная кожа (материал салона), Регулир...",[Диски 17],"[Датчик света, Дневные ходовые огни, Светодиод...","[Подушка безопасности водителя, Подушка безопа...","[Климат-контроль 1-зонный, Кондиционер, Круиз-...","[Центральный замок, Иммобилайзер]",Не указано,Подушка безопасности водителя,Подушка безопасности пассажира,...,Не указано,Не указано,Не указано,Не указано,17,Не указано,Не указано,Центральный замок,Не указано,Иммобилайзер
1,Не указано,[Количество мест: 5],Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,...,Не указано,Не указано,Не указано,Не указано,0,Не указано,Не указано,Не указано,Не указано,Не указано
2,Не указано,[Количество мест: 4],Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,...,Не указано,Не указано,Не указано,Не указано,0,Не указано,Не указано,Не указано,Не указано,Не указано
3,"[AUX, Розетка 12V, Аудиосистема, Аудиоподготовка]","[Кожа (материал салона), Спортивные передние с...",[Диски 19],"[Противотуманные фары, Датчик света, Автоматич...","[Подушка безопасности водителя, Подушка безопа...","[Климат-контроль 1-зонный, Кондиционер, Регули...","[Сигнализация, Центральный замок, Датчик прони...",Не указано,Подушка безопасности водителя,Подушка безопасности пассажира,...,Не указано,Не указано,Не указано,Не указано,19,Сигнализация,Не указано,Центральный замок,Датчик проникновения в салон (датчик объема),Иммобилайзер
4,"[AUX, Розетка 12V, Аудиосистема, Аудиоподготовка]","[Ткань (материал салона), Подогрев передних си...","[Диски 17, Декоративные молдинги]","[Противотуманные фары, Датчик дождя, Автоматич...","[Подушка безопасности водителя, Подушка безопа...","[Климат-контроль 1-зонный, Кондиционер, Регули...","[Центральный замок, Иммобилайзер]",Не указано,Подушка безопасности водителя,Подушка безопасности пассажира,...,Не указано,Не указано,Не указано,Декоративные молдинги,17,Не указано,Не указано,Центральный замок,Не указано,Иммобилайзер


In [ ]:
# Сформируем датасет с опциями из категории "интерьер" применив функцию
# get_interior из модуля myfunc
df_option = pd.DataFrame(df['option_interior'].apply(get_interior).to_list(),
                         columns=['in_color_light',
                                  'in_color_dark',
                                  'in_mat_leather',
                                  'in_mat_combi',
                                  'in_mat_artif_l',
                                  'in_mat_fabric',
                                  'in_mat_velour',
                                  'in_mat_alcantara',
                                  'seat_drive_h_adj',
                                  'seat_front_h_adj',
                                  'seat_drive_e_adj',
                                  'seat_front_e_adj',
                                  'seat_rear_e_adj',
                                  'drive_seat_mem',
                                  'front_seat_mem',
                                  'drive_lumbar_sup',
                                  'front_lumbar_sup',
                                  'front_seat_heat',
                                  'rear_seat_heat',
                                  'front_seat_vent',
                                  'rear_seat_vent',
                                  'seat_count_2',
                                  'seat_count_4',
                                  'seat_count_5',
                                  'seat_count_6',
                                  'seat_count_7',
                                  'seat_count_8',
                                  'seat_count_9',
                                  'sport_seat',
                                  'luke',
                                  'panoramic',
                                  'massage_seat',
                                  'heated_wheel',
                                  'leather_wheel',
                                  'leather_lever',
                                  'black_ceiling',
                                  'rear_armrest',
                                  'third_row_seats',
                                  'fold_rear_seat',
                                  'backrest_fold',
                                  'fold_table',
                                  'in_light',
                                  'f_center_armrest',
                                  'rear_headrest',
                                  'tinted_glass',
                                  'shades_rear_doors',
                                  'shades_rear_window',
                                  'pedal_pads',
                                  'door_sills'
                                  ]
                         )

# Избавимся от лишнего символа '•'
for col in df_option.columns:
    df_option[col] = df_option[col].map(lambda x: ''.join(x.split('•')))

# Преобразуем значения в полях с количеством мест в числа, если опция не была
# указана ставим 0
for i in [2, 4, 5, 6, 7, 8, 9]:
    df_option[f'seat_count_{i}'] = df_option[f'seat_count_{i}'].map(
        lambda x: int(x.split(' ')[-1]) if x != 'Не указано' else 0
        )

# В некоторых объявлениях указно несколько вариантов количества мест, скорее
# всего это модели с дополнительно устанавливаемыми сиденьями. Создадим
# признак с общим количеством мест, взяв их максимальное количество
df_option['seat_count'] = df_option[['seat_count_2',
                                     'seat_count_4',
                                     'seat_count_5',
                                     'seat_count_6',
                                     'seat_count_7',
                                     'seat_count_8',
                                     'seat_count_9'
                                     ]
                                    ].max(axis=1)

# Удалим вспомогательные признаки с конкретным количеством мест
df_option.drop(['seat_count_2',
                'seat_count_4',
                'seat_count_5',
                'seat_count_6',
                'seat_count_7',
                'seat_count_8',
                'seat_count_9'
                ],
               axis=1,
               inplace=True
               )

# Добавляем полученный результат к вспомогательному датасету
df = pd.concat([df, df_option], axis=1)

# Проверяем результат
display(df.head())


,option_media,option_interior,option_exterior,option_visibility,option_safety,option_comfort,option_protection,option_other,airbags_driv,airbags_pass,...,fold_table,in_light,f_center_armrest,rear_headrest,tinted_glass,shades_rear_doors,shades_rear_window,pedal_pads,door_sills,seat_count
0,"[Мультимедиа система с ЖК-экраном, Аудиосистем...","[Искусственная кожа (материал салона), Регулир...",[Диски 17],"[Датчик света, Дневные ходовые огни, Светодиод...","[Подушка безопасности водителя, Подушка безопа...","[Климат-контроль 1-зонный, Кондиционер, Круиз-...","[Центральный замок, Иммобилайзер]",Не указано,Подушка безопасности водителя,Подушка безопасности пассажира,...,Складной столик на спинках передних сидений,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,7
1,Не указано,[Количество мест: 5],Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,...,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,5
2,Не указано,[Количество мест: 4],Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,...,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,4
3,"[AUX, Розетка 12V, Аудиосистема, Аудиоподготовка]","[Кожа (материал салона), Спортивные передние с...",[Диски 19],"[Противотуманные фары, Датчик света, Автоматич...","[Подушка безопасности водителя, Подушка безопа...","[Климат-контроль 1-зонный, Кондиционер, Регули...","[Сигнализация, Центральный замок, Датчик прони...",Не указано,Подушка безопасности водителя,Подушка безопасности пассажира,...,Не указано,Не указано,Передний центральный подлокотник,Не указано,Не указано,Не указано,Не указано,Не указано,Накладки на пороги,2
4,"[AUX, Розетка 12V, Аудиосистема, Аудиоподготовка]","[Ткань (материал салона), Подогрев передних си...","[Диски 17, Декоративные молдинги]","[Противотуманные фары, Датчик дождя, Автоматич...","[Подушка безопасности водителя, Подушка безопа...","[Климат-контроль 1-зонный, Кондиционер, Регули...","[Центральный замок, Иммобилайзер]",Не указано,Подушка безопасности водителя,Подушка безопасности пассажира,...,Не указано,Не указано,Передний центральный подлокотник,Не указано,Не указано,Не указано,Не указано,Не указано,Накладки на пороги,4


In [ ]:
# Сформируем датасет с опциями из категории "прочее" применив функцию
# get_other из модуля myfunc
df_option = pd.DataFrame(df['option_other'].apply(get_other).to_list(),
                         columns=['active_suspension',
                                  'sport_suspension',
                                  'air_suspension',
                                  'full_spare_wheel',
                                  'spare_wheel',
                                  'towbar',
                                  'crankcase_prot'
                                  ]
                         )

# Избавимся от лишнего символа '•'
for col in df_option.columns:
    df_option[col] = df_option[col].map(lambda x: ''.join(x.split('•')))

# Добавляем полученный результат к вспомогательному датасету
df = pd.concat([df, df_option], axis=1)

# Проверяем результат
display(df.head())


,option_media,option_interior,option_exterior,option_visibility,option_safety,option_comfort,option_protection,option_other,airbags_driv,airbags_pass,...,pedal_pads,door_sills,seat_count,active_suspension,sport_suspension,air_suspension,full_spare_wheel,spare_wheel,towbar,crankcase_prot
0,"[Мультимедиа система с ЖК-экраном, Аудиосистем...","[Искусственная кожа (материал салона), Регулир...",[Диски 17],"[Датчик света, Дневные ходовые огни, Светодиод...","[Подушка безопасности водителя, Подушка безопа...","[Климат-контроль 1-зонный, Кондиционер, Круиз-...","[Центральный замок, Иммобилайзер]",Не указано,Подушка безопасности водителя,Подушка безопасности пассажира,...,Не указано,Не указано,7,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано
1,Не указано,[Количество мест: 5],Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,...,Не указано,Не указано,5,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано
2,Не указано,[Количество мест: 4],Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,...,Не указано,Не указано,4,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано
3,"[AUX, Розетка 12V, Аудиосистема, Аудиоподготовка]","[Кожа (материал салона), Спортивные передние с...",[Диски 19],"[Противотуманные фары, Датчик света, Автоматич...","[Подушка безопасности водителя, Подушка безопа...","[Климат-контроль 1-зонный, Кондиционер, Регули...","[Сигнализация, Центральный замок, Датчик прони...",Не указано,Подушка безопасности водителя,Подушка безопасности пассажира,...,Не указано,Накладки на пороги,2,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано
4,"[AUX, Розетка 12V, Аудиосистема, Аудиоподготовка]","[Ткань (материал салона), Подогрев передних си...","[Диски 17, Декоративные молдинги]","[Противотуманные фары, Датчик дождя, Автоматич...","[Подушка безопасности водителя, Подушка безопа...","[Климат-контроль 1-зонный, Кондиционер, Регули...","[Центральный замок, Иммобилайзер]",Не указано,Подушка безопасности водителя,Подушка безопасности пассажира,...,Не указано,Накладки на пороги,4,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано,Не указано


In [ ]:
# Признак equipment содержит очень много уникальных значений, показывающих,
# как наименование комплектации (например, Luxe), так и просто количество
# опций автомобиля. Создадим аналогичный признак посчитав количество опций,
# коорые были указаны в объявлении. Применим функцию get_count_options из
# модуля myfunc
data['count_options'] = df.apply(get_count_options, axis=1)

# Создадим признаки с количеством опций указанных в каждой категории.
# Определим список наименований категорий опций
ls_cat_options = ['media',
                  'interior',
                  'exterior',
                  'visibility',
                  'safety',
                  'comfort',
                  'protection',
                  'other'
                  ]

# В цикле переберем наименования категории опций и для каждой создадим
# признак, применив лямбда функцию, возвращаюшую длину списка (количество
# опций категорий), если опции есть в текущей категории
for cat_options in ls_cat_options:
    data['cnt_' + cat_options] = df['option_' + cat_options]\
        .map(lambda x: len(x) if x != 'Не указано' else 0)

# Проверяем результат
display(data.head(5))


,info,link,url,head,owner,characters,comment,options,parse_date,file_name,...,long,count_options,cnt_media,cnt_interior,cnt_exterior,cnt_visibility,cnt_safety,cnt_comfort,cnt_protection,cnt_other
0,Hyundai Custin\n1.5 л / 170 л.с. / Бензин\nавт...,"<div class=""ListingItem__description""><div cla...",https://auto.ru/cars/used/sale/hyundai/custin/...,"Hyundai Custin, 2024\n15 сентября\n137 (5 сего...",Алексей АВТОНАПРАВЛЕНИЕ\nНет отзывов\nЕкатерин...,Наличие\nВ наличии\nГод выпуска\n2024\nПробег\...,Регистрация в ГИБДД без очереди и получение Го...,Безопасность\n•\nПодушка безопасности водителя...,20-09-2024,chel_data.csv,...,61.402555,48,3,12,1,4,9,17,2,0
1,Hyundai Solaris II\n1.6 л / 123 л.с. / Бензин\...,"<div class=""ListingItem__description""><div cla...",https://auto.ru/cars/used/sale/hyundai/solaris...,"Hyundai Solaris, 2019\n19 сентября\n9 (4 сегод...",Автомобили с пробегом АГАТ на Шефской\n Проспе...,Наличие\nВ наличии\nПоколение\nII\nГод выпуска...,Hyundai Solaris 2019 c пробегом прошел комплек...,Салон\n•\nКоличество мест: 5,20-09-2024,chel_data.csv,...,61.402555,1,0,1,0,0,0,0,0,0
2,Dongfeng DF6\n2.3 л / 163 л.с. / Дизель\nавтом...,"<div class=""ListingItem__description""><div cla...",https://auto.ru/cars/used/sale/dongfeng/df6/11...,"Dongfeng DF6, 2022\nДунфэн Ди-Эф шесть\n2 сент...",Автомобили с пробегом АГАТ на Шефской\n Проспе...,Наличие\nВ наличии\nГод выпуска\n2022\nПробег\...,DongFeng DF6 2022 c пробегом прошел комплексну...,Салон\n•\nКоличество мест: 4,20-09-2024,chel_data.csv,...,61.402555,1,0,1,0,0,0,0,0,0
3,Porsche Cayman S II (981)\n3.4 л / 325 л.с. / ...,"<div class=""ListingItem__description""><div cla...",https://auto.ru/cars/used/sale/porsche/cayman/...,"Porsche Cayman S, 2013\n16 сентября\n583 (18 с...","Частное лицо\nЕкатеринбургМосковская улица, 192",Наличие\nВ наличии\nПоколение\nII (981)\nГод в...,Автомобиль в идеальном состоянии\nСделано то\n...,Безопасность\n•\nПодушка безопасности водителя...,20-09-2024,chel_data.csv,...,61.402555,42,4,7,1,7,8,11,4,0
4,Volkswagen Passat CC I Рестайлинг\n1.8 л / 152...,"<div class=""ListingItem__description""><div cla...",https://auto.ru/cars/used/sale/volkswagen/pass...,"Volkswagen Passat CC, 2012\n19 сентября\n64 (1...",Владимир\nЧелябинскплощадь Ленина,Наличие\nВ наличии\nПоколение\nI Рестайлинг\nГ...,"По кузову живая, сцепление новое, моховик новы...",Безопасность\n•\nПодушка безопасности водителя...,20-09-2024,chel_data.csv,...,61.402555,45,4,8,2,8,10,11,2,0


In [ ]:
# Объединяем вспомогательный и исходный датасет
data = pd.concat([data, df], axis=1)

# Удалим вспомогательные признаки, признаки с большим количеством пропусков,
# уже обработанные и служебные признаки.
data.drop(['info',
           'file_name',
           'link',
           'url',
           'id',
           'owner',
           'comment',
           'parse_date',
           'head',
           'characters',
           'equipment',
           'possession',
           'exchange',
           'vin',
           'state_number',
           'eng_vol',
           'engine',
           'options',
           'option_media',
           'option_interior',
           'option_exterior',
           'option_visibility',
           'option_safety',
           'option_comfort',
           'option_protection',
           'option_other'
           ], axis=1, inplace=True)

# Посмотрим на результат
display(f'Размер итогового датафрейма: {data.shape}')

# Записываем в файл
data.to_csv('./data/auto_data.csv', index=False)


'Размер итогового датафрейма: (35846, 197)'